In [88]:
import datetime as dt
from bs4 import BeautifulSoup
import requests
import re
import time
import random
import pandas as pd
import os
import numpy as np

In [89]:
master_df = pd.read_csv('week_4.csv')

In [90]:
master_df.head(15)

,Pos,Name,Salary,TeamAbbrev,AvgPointsPerGame,Projected Points,dk_pos_sal_avg_points
0,WR,DeAndre Hopkins,8500,ARI,26.53,20.6,19.347368
1,QB,Lamar Jackson,8100,BAL,20.08,23.8,24.040000
2,RB,Alvin Kamara,8000,NO,36.60,23.9,19.637500
3,QB,Russell Wilson,7800,SEA,36.33,23.5,9.320000
4,RB,Ezekiel Elliott,7800,DAL,22.90,21.3,20.421053
5,WR,Michael Thomas,7600,NO,4.70,15.3,15.870909
6,RB,Dalvin Cook,7600,MIN,22.93,19.2,17.600000
7,QB,Josh Allen,7300,BUF,35.64,22.6,19.996000
8,QB,Dak Prescott,7200,DAL,31.64,22.7,18.897500
9,RB,Austin Ekeler,7100,LAC,19.93,18.1,16.438462


In [91]:
master_df['val_above_sal'] = master_df['Projected Points'] - master_df['dk_pos_sal_avg_points']

In [92]:
master_df['Points Per Dollar'] = master_df['Projected Points'] / \
    master_df['Salary']
master_df['Points Per 1K'] = master_df['Projected Points'] / 1000
master_df['Value'] = master_df['Projected Points'] / \
    (master_df['Salary']/1000)

In [93]:
master_df

,Pos,Name,Salary,TeamAbbrev,AvgPointsPerGame,Projected Points,dk_pos_sal_avg_points,val_above_sal,Points Per Dollar,Points Per 1K,Value
0,WR,DeAndre Hopkins,8500,ARI,26.53,20.6,19.347368,1.252632,0.002424,0.0206,2.423529
1,QB,Lamar Jackson,8100,BAL,20.08,23.8,24.040000,-0.240000,0.002938,0.0238,2.938272
2,RB,Alvin Kamara,8000,NO,36.60,23.9,19.637500,4.262500,0.002987,0.0239,2.987500
3,QB,Russell Wilson,7800,SEA,36.33,23.5,9.320000,14.180000,0.003013,0.0235,3.012821
4,RB,Ezekiel Elliott,7800,DAL,22.90,21.3,20.421053,0.878947,0.002731,0.0213,2.730769
...,...,...,...,...,...,...,...,...,...,...,...
398,TE,Deon Yelder,2500,KC,0.00,0.3,1.870506,-1.570506,0.000120,0.0003,0.120000
399,DST,Texans,2500,HOU,1.67,6.0,5.530973,0.469027,0.002400,0.0060,2.400000
400,DST,Browns,2300,CLE,6.00,6.0,5.700935,0.299065,0.002609,0.0060,2.608696
401,DST,Lions,2200,DET,1.67,4.7,5.894737,-1.194737,0.002136,0.0047,2.136364


In [94]:
good_choices = master_df[(master_df['val_above_sal']>0) & (master_df['Projected Points']>11)]

In [95]:
good_choices.sort_values(by=['Pos','val_above_sal'], ascending=False)

,Pos,Name,Salary,TeamAbbrev,AvgPointsPerGame,Projected Points,dk_pos_sal_avg_points,val_above_sal,Points Per Dollar,Points Per 1K,Value
60,WR,DJ Moore,5600,CAR,13.63,15.9,11.217391,4.682609,0.002839,0.0159,2.839286
31,WR,Robert Woods,6300,LAR,17.87,16.6,12.881270,3.718730,0.002635,0.0166,2.634921
11,WR,Tyler Lockett,7000,SEA,25.63,19.1,15.404651,3.695349,0.002729,0.0191,2.728571
49,WR,Odell Beckham Jr.,5800,CLE,10.83,15.3,12.025373,3.274627,0.002638,0.0153,2.637931
48,WR,Terry McLaurin,5800,WAS,17.40,15.1,12.025373,3.074627,0.002603,0.0151,2.603448
75,WR,Justin Jefferson,5200,MIN,15.17,13.4,10.608732,2.791268,0.002577,0.0134,2.576923
55,WR,DeVante Parker,5700,MIA,12.30,16.0,13.357143,2.642857,0.002807,0.0160,2.807018
67,WR,Robby Anderson,5400,CAR,20.27,13.2,10.844776,2.355224,0.002444,0.0132,2.444444
19,WR,Amari Cooper,6700,DAL,18.23,17.9,15.594340,2.305660,0.002672,0.0179,2.671642
43,WR,Will Fuller V,5900,HOU,12.53,14.5,12.243077,2.256923,0.002458,0.0145,2.457627


In [109]:
Tuesday = master_df[master_df['Name'].isin(['Texans ','Russell Wilson',
                'David Johnson', 'Mike Davis', 'DK Metcalf', 
                'Tyler Lockett', "Damiere Byrd", 'Mike Gesicki', "Clyde Edwards-Helaire"])]
Wednesday = master_df[master_df['Name'].isin(['Browns ','Ryan Fitzpatrick',
                'Alvin Kamara', 'Ezekiel Elliott', 'DeVante Parker', 
                'Tyler Lockett', "Damiere Byrd", 'T.J. Hockenson', "Kenyan Drake"])]
Thursday = master_df[master_df['Name'].isin(['Dolphins ','Ryan Fitzpatrick',
                'Alvin Kamara', 'Clyde Edwards-Helaire', 'DeVante Parker', 
                'Tyler Lockett', "Brandin Cooks", 'Darren Waller', "Mike Davis"])]
Friday = master_df[master_df['Name'].isin(['Dolphins ','Ryan Fitzpatrick',
                'Alvin Kamara', 'Ezekiel Elliott', 'DeVante Parker', 
                'DJ Moore', "Brandin Cooks", 'Darren Waller', "Mike Davis"])]

In [110]:
print(Tuesday['Projected Points'].sum(),Tuesday['Salary'].sum())

133.4 49900


In [111]:
print(Wednesday['Projected Points'].sum(),Wednesday['Salary'].sum())

138.10000000000002 50000


In [112]:
print(Thursday['Projected Points'].sum(),Thursday['Salary'].sum())

145.3 49900


In [113]:
print(Friday['Projected Points'].sum(),Friday['Salary'].sum())

145.60000000000002 49900
